In [ ]:
#Exrtracting spectral and VIs and spatial features

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio

# Output parent directory for maps in PNG format
ndvi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/NDVI_maps/'
gndvi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/GNDVI_maps/'
dvi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/DVI_maps/'
evi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/EVI_maps/'
evi2_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/EVI2_maps/'
mari_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/MARI_maps/'
chlre_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/CHLRE_maps/'
chlgr_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/CHLGR_maps/'
savi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/SAVI_maps/'
savi2_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/SAVI2_maps/'
msavi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/MSAVI_maps/'
osavi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/OSAVI_maps/'
tsavi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/TSAVI_maps/'
msr_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/MSR_maps/'
mcari_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/MCARI_maps/'
mcaridiosavi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/MCARIDIOSAVI_maps/'
pvi_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/PVI_maps/'
sr_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/SR_maps/'
tci_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/TCI_maps/'
wdri_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/wdri_maps/'
egreen_output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/egreen_maps/'

# Output directory for the shared CSV file
csv_output_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/'
os.makedirs(csv_output_directory, exist_ok=True)

# Create the output directories if they don't exist
os.makedirs(ndvi_output_parent_directory, exist_ok=True)
os.makedirs(gndvi_output_parent_directory, exist_ok=True)
os.makedirs(dvi_output_parent_directory, exist_ok=True)
os.makedirs(evi_output_parent_directory, exist_ok=True)
os.makedirs(evi2_output_parent_directory, exist_ok=True)
os.makedirs(mari_output_parent_directory, exist_ok=True)
os.makedirs(chlre_output_parent_directory, exist_ok=True)
os.makedirs(chlgr_output_parent_directory, exist_ok=True)
os.makedirs(savi_output_parent_directory, exist_ok=True)
os.makedirs(savi2_output_parent_directory, exist_ok=True)
os.makedirs(msavi_output_parent_directory, exist_ok=True)
os.makedirs(osavi_output_parent_directory, exist_ok=True)
os.makedirs(tsavi_output_parent_directory, exist_ok=True)
os.makedirs(msr_output_parent_directory, exist_ok=True)
os.makedirs(mcari_output_parent_directory, exist_ok=True)
os.makedirs(mcaridiosavi_output_parent_directory, exist_ok=True)
os.makedirs(pvi_output_parent_directory, exist_ok=True)
os.makedirs(sr_output_parent_directory, exist_ok=True)
os.makedirs(tci_output_parent_directory, exist_ok=True)
os.makedirs(wdri_output_parent_directory, exist_ok=True)
os.makedirs(egreen_output_parent_directory, exist_ok=True)

# Initialize a list to store DataFrames for each location
data_list = []
vegetation_fractions_list = []

# Loop through all locations (assuming you have 240 locations)
for location_index in range(1, 241):
    location_name = f'location{location_index}'  # Location name based on the index
    
    # Select the stacked image for the current location
    stacked_image = LUM_masked_maps[location_index - 1]

    # Extract the Red, NIR, and Blue bands from the stacked image
    blue = stacked_image[0]
    green = stacked_image[1]
    red = stacked_image[2]
    rededge = stacked_image[3]
    nir = stacked_image[4]


    # Constants for EVI calculation
    L = 0.16  # Canopy background adjustment; gsavi
    x=0.08 # atsavi
    a=1.43 # atsavi, savi2
    b=0.01 # atsavi, savi2
    C1 = 6.0  # Coefficient of aerosol resistance for red band
    C2 = 7.5  # Coefficient of aerosol resistance for NIR band
    G = 2.5  # Gain factor
    dnir = 0.02
    dred = 0.0063


    num_vegetation_pixels = np.count_nonzero(~np.isnan(stacked_image))
    total_pixels = np.prod(stacked_image.shape)

    # Calculate VIs
    masked_ndvi = (nir - red) / (nir + red) # ndvi
    masked_gndvi =(nir - green) / (nir + green)
    masked_dvi= (nir - red)
    masked_evi = (2.5*(nir-red))/((nir+(6*red)-(7.5*blue))+1) #evi
    masked_evi2 = (2.5*(nir-red))/((nir+(2.4*red))+1)
    masked_mari = (((1/green)-(1/rededge))*nir)
    masked_chlre= (nir/rededge)-1
    masked_chlgr= (nir/green)-1 
    masked_savi= (1.5*(nir - red))/(nir+red+0.5) # SAVI
    masked_savi2= nir/(red+(b/a)) # SAVI2
    masked_msavi= ((2*nir)+1-np.sqrt(((2*nir)+1)**2-(8*(nir-red))))/2 # MSAVI
    masked_osavi=(1+L)*((nir-green)/(nir+green+L)) #OSAVI 
    masked_tsavi = ((a*(nir-(a*red)-b))/((a*nir)+red-(a*b)+(x*(1+a**2)))) #or TSAVI
    masked_msr = ((nir/red)-1)/np.sqrt(((nir/red)-1))
    masked_mcari = ((rededge - red) - 0.2 * (rededge - green)) * (rededge / red)
    masked_mcaridiosavi = masked_mcari/masked_osavi
    masked_pvi = (nir-(a*red)-b)/(np.sqrt(1+a**2))
    masked_sr=nir/red
    masked_wdri= (0.3*(nir-red))/(0.3*(nir+red))
    vegetation_fraction = num_vegetation_pixels / total_pixels

    Plot and save the masked NDVI as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_ndvi, cmap='Blues')  # Set vmin and vmax for color scale
    plt.colorbar(label='NDVI', shrink=0.2)
    plt.title(f'NDVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the NDVI map of the current location
    ndvi_output_directory = os.path.join(ndvi_output_parent_directory)
    os.makedirs(ndvi_output_directory, exist_ok=True)
    # Save the NDVI map as a PNG image
    ndvi_output_image_path = os.path.join(ndvi_output_directory, f'{location_name}_NDVI.png')
    plt.savefig(ndvi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    
    # Plot and save the masked GNDVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_gndvi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='GNDVI', shrink=0.2)
    plt.title(f'GNDVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the EVI map of the current location
    gndvi_output_directory = os.path.join(gndvi_output_parent_directory)
    os.makedirs(gndvi_output_directory, exist_ok=True)
    # Save the EVI map as a PNG image
    gndvi_output_image_path = os.path.join(gndvi_output_directory, f'{location_name}_gndvi.png')
    plt.savefig(gndvi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot and save the masked DVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_dvi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='DVI', shrink=0.2)
    plt.title(f'DVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the DVI map of the current location
    dvi_output_directory = os.path.join(dvi_output_parent_directory)
    os.makedirs(dvi_output_directory, exist_ok=True)
    # Save the DVI map as a PNG image
    dvi_output_image_path = os.path.join(dvi_output_directory, f'{location_name}_DVI.png')
    plt.savefig(dvi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot and save the masked EVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_evi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='EVI', shrink=0.2)
    plt.title(f'EVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the EVI map of the current location
    evi_output_directory = os.path.join(evi_output_parent_directory)
    os.makedirs(evi_output_directory, exist_ok=True)
    # Save the EVI map as a PNG image
    evi_output_image_path = os.path.join(evi_output_directory, f'{location_name}_EVI.png')
    plt.savefig(evi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot and save the masked EVI2 maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_evi2, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='EVI2', shrink=0.2)
    plt.title(f'EVI2 Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the EVI2 map of the current location
    evi2_output_directory = os.path.join(evi2_output_parent_directory)
    os.makedirs(evi2_output_directory, exist_ok=True)
    # Save the EVI2 map as a PNG image
    evi2_output_image_path = os.path.join(evi2_output_directory, f'{location_name}_EVI2.png')
    plt.savefig(evi2_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # Plot and save the masked MARI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_mari, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='MARI', shrink=0.2)
    plt.title(f'MARI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the MARI map of the current location
    mari_output_directory = os.path.join(mari_output_parent_directory)
    os.makedirs(mari_output_directory, exist_ok=True)
    # Save the MARI map as a PNG image
    mari_output_image_path = os.path.join(mari_output_directory, f'{location_name}_MARI.png')
    plt.savefig(mari_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # Plot and save the masked CHLRE maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_chlre, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='CHLRE', shrink=0.2)
    plt.title(f'CHLRE Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the CHLRE map of the current location
    chlre_output_directory = os.path.join(chlre_output_parent_directory)
    os.makedirs(chlre_output_directory, exist_ok=True)
    # Save the CHLRE map as a PNG image
    chlre_output_image_path = os.path.join(chlre_output_directory, f'{location_name}_CHLRE.png')
    plt.savefig(chlre_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot and save the masked CHLGR maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_chlgr, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='CHLGR', shrink=0.2)
    plt.title(f'CHLGR Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the CHLGR map of the current location
    chlgr_output_directory = os.path.join(chlgr_output_parent_directory)
    os.makedirs(chlgr_output_directory, exist_ok=True)
    # Save the CHLGR map as a PNG image
    chlgr_output_image_path = os.path.join(chlgr_output_directory, f'{location_name}_CHLGR.png')
    plt.savefig(chlgr_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked SAVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_savi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='SAVI', shrink=0.2)
    plt.title(f'SAVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the SAVI map of the current location
    savi_output_directory = os.path.join(savi_output_parent_directory)
    os.makedirs(savi_output_directory, exist_ok=True)
    # Save the SAVI map as a PNG image
    savi_output_image_path = os.path.join(savi_output_directory, f'{location_name}_SAVI.png')
    plt.savefig(savi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked SAVI2 maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_savi2, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='SAVI2', shrink=0.2)
    plt.title(f'SAVI2 Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the SAVI2 map of the current location
    savi2_output_directory = os.path.join(savi2_output_parent_directory)
    os.makedirs(savi2_output_directory, exist_ok=True)
    # Save the SAVI2 map as a PNG image
    savi2_output_image_path = os.path.join(savi2_output_directory, f'{location_name}_SAVI2.png')
    plt.savefig(savi2_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked MSAVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_msavi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='MSAVI', shrink=0.2)
    plt.title(f'MSAVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the MSAVI map of the current location
    msavi_output_directory = os.path.join(msavi_output_parent_directory)
    os.makedirs(msavi_output_directory, exist_ok=True)
    # Save the MSAVI map as a PNG image
    msavi_output_image_path = os.path.join(msavi_output_directory, f'{location_name}_MSAVI.png')
    plt.savefig(msavi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked OSAVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_osavi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='OSAVI', shrink=0.2)
    plt.title(f'OSAVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the OSAVI map of the current location
    osavi_output_directory = os.path.join(osavi_output_parent_directory)
    os.makedirs(osavi_output_directory, exist_ok=True)
    # Save the OSAVI map as a PNG image
    osavi_output_image_path = os.path.join(osavi_output_directory, f'{location_name}_OSAVI.png')
    plt.savefig(osavi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked TSAVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_tsavi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='TSAVI', shrink=0.2)
    plt.title(f'TSAVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the TSAVI map of the current location
    tsavi_output_directory = os.path.join(tsavi_output_parent_directory)
    os.makedirs(tsavi_output_directory, exist_ok=True)
    # Save the TSAVI map as a PNG image
    tsavi_output_image_path = os.path.join(tsavi_output_directory, f'{location_name}_TSAVI.png')
    plt.savefig(tsavi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plot and save the masked MSR maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_msr, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='MSR', shrink=0.2)
    plt.title(f'MSR Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the MSR map of the current location
    msr_output_directory = os.path.join(msr_output_parent_directory)
    os.makedirs(msr_output_directory, exist_ok=True)
    # Save the MSR map as a PNG image
    msr_output_image_path = os.path.join(msr_output_directory, f'{location_name}_MSR.png')
    plt.savefig(msr_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()


    # Plot and save the masked MCARI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_mcari, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='MCARI', shrink=0.2)
    plt.title(f'MCARI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the MCARI map of the current location
    mcari_output_directory = os.path.join(mcari_output_parent_directory)
    os.makedirs(mcari_output_directory, exist_ok=True)
    # Save the MCARI map as a PNG image
    mcari_output_image_path = os.path.join(mcari_output_directory, f'{location_name}_MCARI.png')
    plt.savefig(mcari_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked MCARIDIOSAVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_mcaridiosavi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='MCARIDIOSAVI', shrink=0.2)
    plt.title(f'MCARIDIOSAVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the MCARIDIOSAVI map of the current location
    mcaridiosavi_output_directory = os.path.join(mcaridiosavi_output_parent_directory)
    os.makedirs(mcaridiosavi_output_directory, exist_ok=True)
    # Save the MCARIDIOSAVI map as a PNG image
    mcaridiosavi_output_image_path = os.path.join(mcaridiosavi_output_directory, f'{location_name}_MCARIDIOSAVI.png')
    plt.savefig(mcaridiosavi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked PVI maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_pvi, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='PVI', shrink=0.2)
    plt.title(f'PVI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the PVI map of the current location
    pvi_output_directory = os.path.join(pvi_output_parent_directory)
    os.makedirs(pvi_output_directory, exist_ok=True)
    # Save the PVI map as a PNG image
    pvi_output_image_path = os.path.join(pvi_output_directory, f'{location_name}_PVI.png')
    plt.savefig(pvi_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

    # Plot and save the masked SR maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_sr, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='SR', shrink=0.2)
    plt.title(f'SR Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the SR map of the current location
    sr_output_directory = os.path.join(sr_output_parent_directory)
    os.makedirs(sr_output_directory, exist_ok=True)
    # Save the SR map as a PNG image
    sr_output_image_path = os.path.join(sr_output_directory, f'{location_name}_SR.png')
    plt.savefig(sr_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()
    
   

    #Plot and save the masked SR maps as PNG images
    plt.figure(figsize=(8, 6))
    plt.imshow(masked_wdri, cmap='Greens')  # Set vmin and vmax for color scale
    plt.colorbar(label='WDRI', shrink=0.2)
    plt.title(f'WDRI Map for {location_name} with Soil Mask')
    plt.axis('off')
    # Create a directory for the WDRI map of the current location
    wdri_output_directory = os.path.join(wdri_output_parent_directory)
    os.makedirs(wdri_output_directory, exist_ok=True)
    # Save the WDRI map as a PNG image
    wdri_output_image_path = os.path.join(wdri_output_directory, f'{location_name}_WDRI.png')
    plt.savefig(wdri_output_image_path, dpi=300, bbox_inches='tight')
    plt.close()

   


    # Calculate the average NDVI and EVI values for the current location (excluding np.nan values)
    average_ndvi = np.nanmean(masked_ndvi)
    average_gndvi = np.nanmean(masked_gndvi)
    average_dvi = np.nanmean(masked_dvi)
    average_evi = np.nanmean(masked_evi)
    average_evi2 = np.nanmean(masked_evi2)
    average_mari = np.nanmean(masked_mari)
    average_chlre = np.nanmean(masked_chlre)
    average_chlgr = np.nanmean(masked_chlgr)
    average_savi = np.nanmean(masked_savi)
    average_savi2 = np.nanmean(masked_savi2)
    average_msavi = np.nanmean(masked_msavi)
    average_osavi = np.nanmean(masked_osavi)
    average_tsavi = np.nanmean(masked_tsavi)
    average_msr = np.nanmean(masked_msr)
    average_mcari = np.nanmean(masked_mcari)
    average_mcaridiosavi = np.nanmean(masked_mcaridiosavi)
    average_pvi = np.nanmean(masked_pvi)
    average_sr = np.nanmean(masked_sr)
    average_wdri = np.nanmean(masked_wdri)
    

    # Create a DataFrame for the current location
    data_location = pd.DataFrame({'Location': [location_index], 'NDVI': [average_ndvi], 'GNDVI': [average_gndvi],
                                  'DVI': [average_dvi], 'EVI': [average_evi], 'EVI2': [average_evi2]
                                  ,'MARI': [average_mari], 'CHLRE': [average_chlre],
                                  'CHLGR': [average_chlgr], 'SAVI': [average_savi], 'SAVI2': [average_savi2],
                                  'MSAVI': [average_msavi], 'OSAVI': [average_osavi], 'TSAVI': [average_tsavi], 
                                  'MSR': [average_msr], 'MCARI': [average_mcari],
                                  'MCARI/OSAVI': [average_mcaridiosavi], 'PVI': [average_pvi], 
                                  'SR': [average_sr],'WDRI': [average_wdri], 
                                  'VF': [vegetation_fraction]})
    data_list.append(data_location)

# Concatenate all DataFrames into a single DataFrame
combined_data = pd.concat(data_list, ignore_index=True)

# Save the combined DataFrame as a CSV file
csv_output_path = os.path.join(csv_output_directory, '6_15_2020_LUM_average_vis_values.csv')
combined_data.to_csv(csv_output_path, index=False)

# Print the combined DataFrame
print("Average VI's values for each location:")
print(combined_data)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# CSV file containing average NDVI and EVI values
csv_file_path = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/6_15_2020_LUM_average_vis_values.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(csv_file_path)

# Create a vertical box plot of both average NDVI and EVI values
plt.figure(figsize=(8, 6))
plt.boxplot([data['NDVI'], data['GNDVI'], data['DVI'], data['EVI'], data['EVI2'], 
             data['MARI'], data['CHLRE'],data['CHLGR'], data['SAVI'], 
             data['SAVI2'], data['MSAVI'], data['OSAVI'], data['TSAVI'], data['MSR'],
            data['MCARI'],data['MCARI/OSAVI'],data['PVI'],data['SR'],
            data['WDRI'],data['VF']]
            , labels=['NDVI', 'GNDVI','DVI','EVI','EVI2','MARI','CHLRE','CHLGR','SAVI','SAVI2','MSAVI','OSAVI','TSAVI',
                      'MSR','MCARI','MCARI/OSAVI','PVI','SR','WDRI','VF'])
plt.title('Vertical Box Plot of AverageValues')
plt.ylabel('Average Value')
plt.tight_layout()

# Save the box plot as an image
output_image_path = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM_VI/vertical_average__box_plot.png'
plt.savefig(output_image_path, dpi=300, bbox_inches='tight')
plt.show()